In [2]:
# based on https://towardsdatascience.com/selenium-tutorial-scraping-glassdoor-com-in-10-minutes-3d0915c6d905

In [ ]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import os.path
from datetime import date
import pandas as pd

In [75]:
import cred_

def sign_in(driver):

    url = 'https://www.glassdoor.com/profile/login_input.htm'
    driver.get(url)

    email_field = driver.find_element_by_name('username')
    password_field = driver.find_element_by_name('password')
    submit_btn = driver.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys(cred_.username)
    password_field.send_keys(cred_.password)
    submit_btn.click()

    time.sleep(1)

In [61]:
def get_jobs(
    keyword_role,
    keyword_location="Boston,%20MA",
    keyword_age=3,
    num_jobs=1,
    verbose=False,
):

    """Gathers jobs as a dataframe, scraped from Glassdoor"""

    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    sign_in(driver)

    url = (
        "https://www.glassdoor.com/Job/jobs.htm?sc.keyword="
        + keyword_role
        + "&locT=C&locId=1154532&locKeyword="
        + keyword_location
        + "&jobType=all&fromAge="
        + str(keyword_age)
        + "&minSalary=0&includeNoSalaryJobs=true&radius=25&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0"
    )
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:

        # Going through each job in this page
        job_buttons = driver.find_elements_by_class_name(
            "jl"
        )  # jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  
            time.sleep(1)
            collected_successfully = False

            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath(
                        './/div[@class="employerName"]'
                    ).text
                    location = driver.find_element_by_xpath(
                        './/div[@class="location"]'
                    ).text
                    job_title = driver.find_element_by_xpath(
                        './/div[contains(@class, "title")]'
                    ).text
                    job_description = driver.find_element_by_xpath(
                        './/div[@class="jobDescriptionContent desc"]'
                    ).text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath(
                    './/span[@class="gray small salary"]'
                ).text
            except NoSuchElementException:
                salary_estimate = (
                    -1
                )  # You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1  # You need to set a "not found value. It's important."

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            # Going to the Company tab...
            # clicking on this:
            # <div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath(
                    './/div[@class="tab" and @data-tab-type="overview"]'
                ).click()

                try:
                    # <div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    # </div>
                    headquarters = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath(
                        './/div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*'
                    ).text
                except NoSuchElementException:
                    revenue = -1


            except NoSuchElementException:  # Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append(
                {
                    "Job Title": job_title,
                    "Salary Estimate": salary_estimate,
                    "Job Description": job_description,
                    "Rating": rating,
                    "Company Name": company_name.split("\n")[0],
                    "Location": location,
                    "Headquarters": headquarters,
                    "Size": size,
                    "Founded": founded,
                    "Type of ownership": type_of_ownership,
                    "Industry": industry,
                    "Sector": sector,
                    "Revenue": revenue,
                    'Date': str(date.today())
                }
            )
            # add job to jobs

        # Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print(
                "Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(
                    num_jobs, len(jobs)
                )
            )
            break

    return pd.DataFrame(
        jobs
    )  

In [62]:
df = get_jobs(
    keyword_role="\"data+scientist\"+%7C+\"machine+learning\"",
    keyword_location="Boston,%20MA",
    keyword_age=3,
    num_jobs=100,
    verbose=False,
)

Progress: 0/280
Progress: 1/280
Progress: 2/280
Progress: 3/280
Progress: 4/280
Progress: 5/280
Progress: 6/280
Progress: 7/280
Progress: 8/280
Progress: 9/280
Progress: 10/280
Progress: 11/280
Progress: 12/280
Progress: 13/280
Progress: 14/280
Progress: 15/280
Progress: 16/280
Progress: 17/280
Progress: 18/280
Progress: 19/280
Progress: 20/280
Progress: 21/280
Progress: 22/280
Progress: 23/280
Progress: 24/280
Progress: 25/280
Progress: 26/280
Progress: 27/280
Progress: 28/280
Progress: 29/280
Progress: 30/280
Progress: 31/280
Progress: 32/280
Progress: 33/280
Progress: 34/280
Progress: 35/280
Progress: 36/280
Progress: 37/280
Progress: 38/280
Progress: 39/280
Progress: 40/280
Progress: 41/280
Progress: 42/280
Progress: 43/280
Progress: 44/280
Progress: 45/280
Progress: 46/280
Progress: 47/280
Progress: 48/280
Progress: 49/280
Progress: 50/280
Progress: 51/280
Progress: 52/280
Progress: 53/280
Progress: 54/280
Progress: 55/280
Progress: 56/280
Progress: 57/280
Progress: 58/280
Progres

In [64]:
df.to_csv('jobs_data_science.csv', header=False, mode = 'a')

In [66]:
df = get_jobs(
    keyword_role="data analyst",
    keyword_location="Boston,%20MA",
    keyword_age=30,
    num_jobs=150,
    verbose=False,
)

Progress: 0/150
Progress: 1/150
Progress: 2/150
Progress: 3/150
Progress: 4/150
Progress: 5/150
Progress: 6/150
Progress: 7/150
Progress: 8/150
Progress: 9/150
Progress: 10/150
Progress: 11/150
Progress: 12/150
Progress: 13/150
Progress: 14/150
Progress: 15/150
Progress: 16/150
Progress: 17/150
Progress: 18/150
Progress: 19/150
Progress: 20/150
Progress: 21/150
Progress: 22/150
Progress: 23/150
Progress: 24/150
Progress: 25/150
Progress: 26/150
Progress: 27/150
Progress: 28/150
Progress: 29/150
Progress: 30/150
Progress: 31/150
Progress: 32/150
Progress: 33/150
Progress: 34/150
Progress: 35/150
Progress: 36/150
Progress: 37/150
Progress: 38/150
Progress: 39/150
Progress: 40/150
Progress: 41/150
Progress: 42/150
Progress: 43/150
Progress: 44/150
Progress: 45/150
Progress: 46/150
Progress: 47/150
Progress: 48/150
Progress: 49/150
Progress: 50/150
Progress: 51/150
Progress: 52/150
Progress: 53/150
Progress: 54/150
Progress: 55/150
Progress: 56/150
Progress: 57/150
Progress: 58/150
Progres

In [67]:
df.to_csv('jobs_data analyst.csv', header=False, mode = 'a')